In [22]:
#import libraries
from pymongo import MongoClient
from pprint import pprint
from bson.json_util import dumps
import redis
import json
import time
import csv, redis, json
import sys

#this code creat models for our dataset
def prepareModel(columns,rowdata):
    model ={}
    i = 0
    while i < len(rowdata):
        model[columns[i]] = rowdata[i]
        i += 1     
    return model


# read dataset from csv file
def read_csv_data(csv_file,readLineCount):
    columns = []
    dataModel = []
    with open(csv_file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count <= readLineCount:
                if line_count == 0:
                    #print(row)
                    #get colums
                    for i in range(len(row)):
                        columns.append(row[i])
                    line_count += 1
                else:
                    dataModel.append(prepareModel(columns,row))
                    line_count += 1
    return dataModel

#store data to redis
def store_data_to_redis_bulk(conn, data,startIndex,insertLength): 
     #sadd: is the insertion of data set types for the redis
     # If your list is redis data type then use lpush or rpush
    with conn.pipeline(transaction=False) as p:
        for i in  range(startIndex,insertLength):
            key = "data_trump_index_"+str(i)
            value = json.dumps(data[i])
            p.set(key,value)
        p.execute()
        print("all data inserted to redis...")
def delete_all_data_redis_bulk(conn):
    pipe = conn.pipeline()
    count = 0
    for key in conn.scan_iter('*'):
        pipe.delete(key)
        count+=1
    pipe.execute()
    print(" total deleted records:{}".format(count))

def read_all_data_redis_bulk(conn):
    pipe = conn.pipeline()
    count = 0
    for key in conn.scan_iter('*'):
        pipe.get(key)
        count+=1
    data = pipe.execute()
    print("all data length:{}".format(len(data)))
    return data
        
def main():
    lenCounts = [25, 50, 100, 200, 500, 1000, 5000, 10000, 50000,100000]
    times=[]
    times_read=[]
    times_deleted =[]
    for i in range(len(lenCounts)):
        data = read_csv_data("mydataset.csv",lenCounts[i])
        print("Data model info:{}".format(len(data)))
        #print(json.dumps(data[0]))
        #create Redis connection
        conn = redis.Redis(host='myrediscache.xpz4sa.0001.use1.cache.amazonaws.com',port=6379)
        print("Redis connected....")
        #start insert data to redis
        print("Data sending to Redis...")
        ts = time.time()
        store_data_to_redis_bulk(conn,data,0,lenCounts[i])
        times.append(time.time()-ts)
        
        #read redis data bulk
        ts = time.time()
        result=read_all_data_redis_bulk(conn)
        times_read.append(time.time()-ts)
        
        #delete all records
        ts = time.time()
        delete_all_data_redis_bulk(conn)
        times_deleted.append(time.time() - ts)
    #show plot
    print("Insert times:{}".format(times))
    print("Read times:{}".format(times_read))
    print("Deleted times:{}".format(times_deleted))
if '__main__' == __name__:
    main()




Data model info:20
{"created_at": "2020-10-15 00:00:01", "tweet_id": "1.316529221557252e+18", "tweet": "#Elecciones2020 | En #Florida: #JoeBiden dice que #DonaldTrump solo se preocupa por \u00e9l mismo. El dem\u00f3crata fue anfitri\u00f3n de encuentros de electores en #PembrokePines y #Miramar. Clic AQU\u00cd \u2b07\ufe0f\u2b07\ufe0f\u2b07\ufe0f\n\u2800\n\ud83c\udf10https://t.co/qhIWpIUXsT\n_\n#ElSolLatino #yobrilloconelsol https://t.co/6FlCBWf1Mi", "likes": "0.0", "retweet_count": "0.0", "source": "TweetDeck", "user_id": "360666534.0", "user_name": "El Sol Latino News", "user_screen_name": "elsollatinonews", "user_description": "\ud83c\udf10 Noticias de inter\u00e9s para latinos de la costa este de #EEUU\n\u2800\u23f9\ufe0f Facebook e Instagram\n\u2800\ud83c\udfd9\ufe0f Philadelphia: /elsollatinonewspaper\n\u2800\ud83c\udf05 Miami: /elsollatinonewsmiami", "user_join_date": "2011-08-23 15:33:45", "user_followers_count": "1860.0", "user_location": "Philadelphia, PA / Miami, FL", "lat":

ConnectionError: Error 60 connecting to myrediscache.xpz4sa.0001.use1.cache.amazonaws.com:6379. Operation timed out.